<a href="https://colab.research.google.com/github/Rakabi007/Analyzing-NBA-Stadium-Concession-Staffing-Needs-and-Minimizing-the-Time-it-Takes-to-Serve-A-Customer/blob/main/Analyzing_NBA_Stadium_Concession_Staffing_Needs_and_Minimizing_the_Time_it_Takes_to_Serve_A_Customer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Let's do ana analysis based on:

# Date > we can derive day_of_week (Mon–Sun).

# Start time (ET) > start_hour, is_weekend, is_matinee, etc.

# Home team / visitor team > you can flag “marquee” games (Lakers, Celtics, rivalry, etc.).

# Attendance > direct proxy for total potential demand.

# Arena > lets you isolate one venue and build a custom model for that stadium.

# Game length (LOG column) > can be turned into “minutes in building” for throughput assumptions.


# We'll need to synthesize the data below, because we don't have actual data:

# Actual concession sales (hot dogs, beers, etc.)

# Staffing levels per game or per stand

# Within-game time series of sales or queues

In [ ]:
# One way to screen-scrape data...

import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


url = "https://www.basketball-reference.com/leagues/NBA_2024_games.html"
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; RyanConcessionsBot/1.0)"
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "lxml")

table = soup.find("table", id="schedule")
rows = table.find("tbody").find_all("tr")

data = []
for tr in rows:
    # skip header rows inside tbody
    if "class" in tr.attrs and "thead" in tr["class"]:
        continue

    cells = tr.find_all(["th", "td"])
    if not cells:
        continue

    row = {
        "date":           cells[0].get_text(strip=True),
        "start_et":       cells[1].get_text(strip=True),
        "visitor":        cells[2].get_text(strip=True),
        "visitor_pts":    cells[3].get_text(strip=True),
        "home":           cells[4].get_text(strip=True),
        "home_pts":       cells[5].get_text(strip=True),
        "box_score":      cells[6].get_text(strip=True),
        "ot":             cells[7].get_text(strip=True),
        "attendance":     cells[8].get_text(strip=True),
        "log":            cells[9].get_text(strip=True),
        "arena":          cells[10].get_text(strip=True),
        "notes":          cells[11].get_text(strip=True),
    }
    data.append(row)

print(df.shape)
df = pd.DataFrame(data)
display(df.head())
display(df.tail())


In [ ]:
import pandas as pd

# --- Attendance: same as before ---
df["attendance"] = (
    df["attendance"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .replace("", pd.NA)
    .astype("Int64")
)

# --- Parse Date ---
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# --- Game duration "2:17" -> minutes ---
h_m = df["log"].astype(str).str.split(":", expand=True)
df["game_minutes"] = (
    h_m[0].astype(int) * 60 +
    h_m[1].astype(int)
)

# --- Clean Start (ET) and extract hour ---

# 1) Convert to string and strip spaces
start = df["start_et"].astype(str).str.strip()

# 2) Remove rows that are just header repeats or blanks
#    (optional but helpful)
mask_valid = start.str.contains(r"\d", na=False)
start_valid = start.where(mask_valid, None)

# 3) Turn "7:30p" -> "7:30PM", "10:00a" -> "10:00AM"
start_valid = (
    start_valid
    .str.replace(r"a$", "AM", regex=True)
    .str.replace(r"p$", "PM", regex=True)
)

# 4) Parse times
df["Start_hour"] = pd.to_datetime(
    start_valid,
    format="%I:%M%p",
    errors="coerce"          # invalid ones become NaT
).dt.hour

# --- Day of week & weekend flag ---
df["dow"] = df["date"].dt.day_name()
df["is_weekend"] = df["dow"].isin(["Friday", "Saturday", "Sunday"]).astype(int)

display(df.head())

In [ ]:
# Create concessions for all teams.
# Then group by home to see per-team stats.

import numpy as np

df_all = df.copy()

# Night game flag
df_all["night_game"] = (df_all["Start_hour"] >= 18).astype(int)

# Capacity proxy
df_all["attend_k"] = df_all["attendance"] / 1000.0

rng = np.random.default_rng(42)

base_rate_per_fan = 0.35   # avg items per attendee
weekend_uplift = 0.15
night_uplift = 0.10

lam = (
    df_all["attendance"] *
    base_rate_per_fan *
    (1 + weekend_uplift * df_all["is_weekend"] +
       night_uplift     * df_all["night_game"])
)

df_all["lambda_items"] = lam
df_all["items_sold_total"] = rng.poisson(lam)

# Optional breakdown: drinks / food / snacks
df_all["items_drink"] = rng.binomial(df_all["items_sold_total"], 0.40)
remaining = df_all["items_sold_total"] - df_all["items_drink"]
df_all["items_food"] = rng.binomial(remaining, 0.45)
df_all["items_snack"] = remaining - df_all["items_food"]

display(df_all[["date", "home", "attendance", "is_weekend", "night_game", "items_sold_total"]].head())

In [ ]:
team_stats = (
    df_all
    .groupby("home")
    .agg(
        games_played       = ("date", "count"),
        avg_attendance     = ("attendance", "mean"),
        avg_items_sold     = ("items_sold_total", "mean"),
        total_items_sold   = ("items_sold_total", "sum"),
        avg_items_per_fan  = ("items_sold_total", lambda x: x.sum() / df_all.loc[x.index, "attendance"].sum()),
    )
    .sort_values("avg_items_sold", ascending=False)
)

display(team_stats.head(10))

In [ ]:
team_weekend_stats = (
    df_all
    .groupby(["home", "is_weekend"])
    .agg(
        games        = ("date", "count"),
        avg_items    = ("items_sold_total", "mean"),
        avg_attend   = ("attendance", "mean"),
    )
    .reset_index()
)

display(team_weekend_stats.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import PoissonRegressor
from sklearn.ensemble import RandomForestRegressor

# --- Features and target ---
feature_cols = ["attendance", "is_weekend", "night_game", "Start_hour"]
X = df_all[feature_cols]
y = df_all["items_sold_total"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


# --- Random Forest ---
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=6,
    random_state=42
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print("RF MAE:", mean_absolute_error(y_test, y_pred_rf))
print("RF R² :", r2_score(y_test, y_pred_rf))

# Quick look at feature importances from RF
fi = pd.Series(rf.feature_importances_, index=feature_cols).sort_values(ascending=False)
print("\nRF feature importances:")
print(fi)


In [ ]:
# Note:

# attendance ~ 17,000
# items sold ~ attendance∗0.35=6,000

# So an MAE of 205 means:

# 205/6000 ~ 3.4%

# RF R² : 0.86 means the model is making quite accurate predictions and is correct around 86% of the time.









# This could be confused as a Poisson problem, but remember:

# Concessions involves BOTH:
# 1) Total items sold (macro forecasting)

# 2) Arrival processes / queues (micro forecasting)


# These are different prediction problems, but they are connected.

# Total Concession Sales = a forecasting problem (not Poisson)

# “How many beers will we sell tonight?”

# “How many hot dogs do we need to prep?”

# “How much inventory should we stock?”

# Those are big numbers (4,000–12,000 items per game).

# That is not a Poisson process because Poisson is used for small counts per unit time.









# From total-game demand > time profile

# We can define a time-of-game shape

# Assume a 180-minute window:
# 0–29: pre-game (30 min)
# 30–74: 1st half (45 min)
# 75–89: halftime (15 min)
# 90–134: 2nd half (45 min)
# 135–179: post-game (45 min)

# We’ll assign relative weights (how “hot” each minute is):
# pre-game: 0.5
# 1st half: 1.0
# halftime: 2.0
# 2nd half: 1.0
# post-game: 0.3


In [ ]:
# Now, normalize so the weights sum to 1 and scale by items_sold_total.

import numpy as np
import pandas as pd

# --- Add a simple game_id if not already present ---
df_all = df_all.reset_index(drop=True)
df_all["game_id"] = df_all.index

def build_time_shape(total_minutes=180):
    """
    Return a vector of length total_minutes giving relative demand weights per minute.
    """
    shape = np.zeros(total_minutes, dtype=float)

    # Indices are [start, end) in minutes
    # 0-29: pre-game
    shape[0:30] = 0.5

    # 30-74: first half
    shape[30:75] = 1.0

    # 75-89: halftime (peak)
    shape[75:90] = 2.0

    # 90-134: second half
    shape[90:135] = 1.0

    # 135-179: post-game
    shape[135:180] = 0.3

    # Any remaining unassigned minutes (if shape has gaps) we can fill with small value
    shape[shape == 0] = 0.2

    return shape

time_shape_raw = build_time_shape(180)
time_shape = time_shape_raw / time_shape_raw.sum()  # normalize to sum to 1

print("Shape length:", len(time_shape))
print("Shape sum:", time_shape.sum())




In [ ]:
# Build a long time-expanded demand table

rows = []

for _, row in df_all.iterrows():
    game_id = row["game_id"]
    home_team = row["home"]
    arena = row["arena"]
    date = row["date"]
    items_total = row["items_sold_total"]

    # expected items per minute = total * normalized shape
    lam_minutes = items_total * time_shape  # array length 180

    for minute in range(180):
        rows.append({
            "game_id": game_id,
            "date": date,
            "home_team": home_team,
            "arena": arena,
            "minute": minute,                 # minutes from pre-game start
            "lambda_items_min": lam_minutes[minute]
        })

df_time = pd.DataFrame(rows)
df_time.head()




In [ ]:
# Assume stochastic per minute (Poisson) and per stand

rng = np.random.default_rng(123)

# Choose number of stands per arena
N_STANDS = 10

# For simplicity, assume equal share per stand
stand_shares = np.ones(N_STANDS) / N_STANDS

# Add a random realization of items sold per minute and stand
rows = []

for (game_id, grp) in df_time.groupby("game_id"):
    # Get the per-minute lambda for the game
    lam_min = grp["lambda_items_min"].values  # length 180

    for stand_id in range(N_STANDS):
        share = stand_shares[stand_id]
        lam_stand_min = lam_min * share

        # Poisson arrivals per minute for this stand & game
        items_minute = rng.poisson(lam_stand_min)

        # Build rows
        for i, base_row in grp.iterrows():
            minute = int(base_row["minute"])
            rows.append({
                "game_id": game_id,
                "date": base_row["date"],
                "home_team": base_row["home_team"],
                "arena": base_row["arena"],
                "stand_id": stand_id,
                "minute": minute,
                "lambda_stand_min": lam_stand_min[minute],
                "arrivals_min": items_minute[minute],  # customers/items arriving this minute
            })

df_stand_time = pd.DataFrame(rows)

print(df_stand_time.shape)

display(df_stand_time.head())
display(df_stand_time.tail())


# Note:
# We built a time-expanded, stand-expanded dataset for queueing and staffing.
# We started with 54 games and the dataframe here has 97,200 rows. So what happened?!
# 97200 / 54 = 1,800 rows per game
# N_MINUTES = 180 (for a 3-hour window)

# So...
# Games: 54
# Stands: 10
# Minutes per game: 180
# %4 * 10 * 180 = 97200









In [ ]:
import plotly.express as px

game_to_plot = 0
stand_to_plot = 0

sub = df_stand_time[
    (df_stand_time["game_id"] == game_to_plot) &
    (df_stand_time["stand_id"] == stand_to_plot)
]

fig = px.line(
    sub,
    x="minute",
    y="arrivals_min",
    title=f"Arrivals per minute – Game {game_to_plot}, Stand {stand_to_plot}",
    labels={"minute": "Minute", "arrivals_min": "Arrivals per minute"},
)
fig.update_layout(height=600, width=1100)
fig.show()

In [ ]:
import plotly.express as px

shape_avg = (
    df_stand_time
    .groupby("minute")[["lambda_stand_min", "arrivals_min"]]
    .mean()
    .reset_index()
)

fig = px.line(
    shape_avg,
    x="minute",
    y=["lambda_stand_min", "arrivals_min"],
    title="Average per-minute demand profile (all games, all stands)",
    labels={"value": "Customers per minute", "minute": "Minute"},
)

fig.update_layout(
    legend_title_text="Metric",
    height=600,
    width=1100
)

fig.update_layout(height=600, width=1100)
fig.show()
df_stand_time = df_stand_time.drop(columns=["is_weekend"], errors="ignore")

df_stand_time = df_stand_time.merge(
    df_all[["game_id", "is_weekend"]],
    on="game_id",
    how="left"
)


In [ ]:
import plotly.express as px


display(df_stand_time)

shape_wk = (
    df_stand_time
    .groupby(["is_weekend", "minute"])["arrivals_min"]
    .mean()
    .reset_index()
)

# Map 0/1 to labels for nicer plotting
shape_wk["day_type"] = shape_wk["is_weekend"].map({0: "Weekday", 1: "Weekend"})

fig = px.line(
    shape_wk,
    x="minute",
    y="arrivals_min",
    color="day_type",
    title="Weekday vs Weekend demand profile",
    labels={
        "minute": "Minute",
        "arrivals_min": "Avg Arrivals per Minute",
        "day_type": "Day Type"
    },
)

fig.update_layout(height=600, width=1100)
fig.show()

In [ ]:
import plotly.express as px

g = 0  # game_id to inspect
sub = df_stand_time[df_stand_time["game_id"] == g]

pivot = sub.pivot_table(
    index="stand_id",
    columns="minute",
    values="arrivals_min",
    aggfunc="sum"
)

fig = px.imshow(
    pivot,
    aspect="auto",
    color_continuous_scale="Viridis",
    labels={"color": "Arrivals/min"},
    title=f"Heatmap of arrivals – Game {g}"
)

fig.update_layout(
    height=600,
    width=1100,
    xaxis_title="Minute",
    yaxis_title="Stand ID",
)

fig.show()

In [ ]:

import numpy as np
import pandas as pd

# If simulate_queue_discrete is already defined in your notebook, skip this block.
def simulate_queue_discrete(arrivals, servers=3, mu_per_server=0.5, rng=None):
    """
    arrivals: 1D array-like of arrivals per minute.
    servers: number of servers (staff).
    mu_per_server: avg customers served per minute by each server.
    """
    arrivals = np.array(arrivals, dtype=float)
    T = len(arrivals)
    if rng is None:
        rng = np.random.default_rng()

    queue = 0.0
    queue_len = np.zeros(T, dtype=float)
    served = np.zeros(T, dtype=float)

    for t in range(T):
        queue += arrivals[t]
        service_capacity = rng.poisson(servers * mu_per_server)
        n_served = min(queue, service_capacity)
        queue -= n_served
        queue_len[t] = queue
        served[t] = n_served

    avg_queue = queue_len.mean()
    total_served = served.sum()
    lam_eff = total_served / T if T > 0 else 0.0

    if lam_eff > 0:
        avg_wait = avg_queue / lam_eff  # Little's Law
    else:
        avg_wait = np.nan

    return {
        "avg_queue": avg_queue,
        "avg_wait": avg_wait,
        "lambda_eff": lam_eff,
    }

# -----------------------------
# Build parameter grid
# -----------------------------
servers_list = list(range(2, 16))                # 2–15 servers
speed_list   = np.round(np.linspace(0.5, 2.0, 16), 2)  # 0.5–2.0 customers/min

grid_results = []

for s in servers_list:
    for mu in speed_list:
        res = simulate_queue_discrete(
            arrivals,
            servers=s,
            mu_per_server=mu,
            rng=np.random.default_rng(123)  # fixed seed for reproducibility
        )
        grid_results.append({
            "servers": s,
            "mu_per_server": mu,
            "avg_wait_min": res["avg_wait"]
        })

df_grid = pd.DataFrame(grid_results)
display(df_grid.head())

# Pivot so rows = speed, columns = servers, values = avg wait
heat = df_grid.pivot_table(
    index="mu_per_server",
    columns="servers",
    values="avg_wait_min"
)

# Optional: clip crazy-long waits to make the color scale readable
heat_clipped = heat.clip(upper=120)  # cap at 120 min wait for display

display(heat_clipped.head())


import plotly.express as px

fig = px.imshow(
    heat_clipped,
    labels=dict(
        x="Number of servers",
        y="Service speed per server (customers/min)",
        color="Avg wait (min)"
    ),
    aspect="auto"
)

fig.update_layout(
    title="Average Wait Time Heatmap: Servers vs Service Speed",
    height=600,
    width=900
)

fig.show()


In [ ]:
# Queueing / staffing simulator

# Next, we’ll simulate in discrete time and approximate:

# queue length over time

# throughput

# average waiting time (via Little’s Law: W ≈ L / λ)

def simulate_queue_discrete(arrivals, servers=3, mu_per_server=0.5, rng=None):
    """
    arrivals: 1D array-like of arrivals per minute (Poisson or given).
    servers: number of servers (staff).
    mu_per_server: average customers served per minute by each server.

    Returns dict with:
        queue_len: array of queue length each minute
        served: array of customers served each minute
        avg_queue: average queue length
        avg_wait: approximate average waiting time (minutes) via Little's Law
    """
    arrivals = np.array(arrivals, dtype=float)
    T = len(arrivals)
    if rng is None:
        rng = np.random.default_rng()

    queue = 0
    queue_len = np.zeros(T, dtype=float)
    served = np.zeros(T, dtype=float)

    for t in range(T):
        # Step 1: arrivals join the queue
        queue += arrivals[t]

        # Step 2: service capacity this minute (random)
        # Poisson(server * mu) is a decent approximation for completions per minute.
        service_capacity = rng.poisson(servers * mu_per_server)

        # How many can we serve?
        n_served = min(queue, service_capacity)

        queue -= n_served
        queue_len[t] = queue
        served[t] = n_served

    avg_queue = queue_len.mean()
    total_served = served.sum()
    T_minutes = T

    # Effective throughput per minute
    lam_eff = total_served / T_minutes if T_minutes > 0 else 0.0

    if lam_eff > 0:
        avg_wait = avg_queue / lam_eff  # Little's Law W = L / λ
    else:
        avg_wait = np.nan

    return {
        "queue_len": queue_len,
        "served": served,
        "avg_queue": avg_queue,
        "avg_wait": avg_wait,
        "lambda_eff": lam_eff
    }


In [ ]:
# Finally, run simulator for one game & stand

# Pick a single high-attendance home game
game_idx = df_all["attendance"].idxmax()
game_id = df_all.loc[game_idx, "game_id"]
print("Simulating game_id:", game_id, "home:", df_all.loc[game_idx, "home"])

# Take one stand at that game, e.g. stand 0
stand_id = 0

sub = df_stand_time[
    (df_stand_time["game_id"] == game_id) &
    (df_stand_time["stand_id"] == stand_id)
].sort_values("minute")

arrivals = sub["arrivals_min"].values  # length ~180

print("Total arrivals for this stand:", arrivals.sum())

# Try different staffing levels
for servers in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    result = simulate_queue_discrete(
        arrivals,
        servers=servers,
        mu_per_server=0.3,   # each staff serves ~0.9 customers/min ≈ 1 every 1.7 min
        rng=np.random.default_rng(123)
    )

    print(f"\nServers = {servers}")
    print(f"  Avg queue length: {result['avg_queue']:.2f}")
    print(f"  Approximated avg wait (min): {result['avg_wait']:.2f}")
    print(f"  Effective throughput λ_eff: {result['lambda_eff']:.2f} customers/min")






Simulating game_id: 10 home: Chicago Bulls
Total arrivals for this stand: 861

Servers = 2
  Avg queue length: 427.59
  Approximated avg wait (min): 754.57
  Effective throughput λ_eff: 0.57 customers/min

Servers = 3
  Avg queue length: 403.26
  Approximated avg wait (min): 493.78
  Effective throughput λ_eff: 0.82 customers/min

Servers = 4
  Avg queue length: 384.02
  Approximated avg wait (min): 361.91
  Effective throughput λ_eff: 1.06 customers/min

Servers = 5
  Avg queue length: 354.12
  Approximated avg wait (min): 248.02
  Effective throughput λ_eff: 1.43 customers/min

Servers = 6
  Avg queue length: 328.16
  Approximated avg wait (min): 192.41
  Effective throughput λ_eff: 1.71 customers/min

Servers = 7
  Avg queue length: 294.03
  Approximated avg wait (min): 138.19
  Effective throughput λ_eff: 2.13 customers/min

Servers = 8
  Avg queue length: 264.21
  Approximated avg wait (min): 107.11
  Effective throughput λ_eff: 2.47 customers/min

Servers = 9
  Avg queue length: 238.46
  Approximated avg wait (min): 86.19
  Effective throughput λ_eff: 2.77 customers/min

Servers = 10
  Avg queue length: 210.59
  Approximated avg wait (min): 68.92
  Effective throughput λ_eff: 3.06 customers/min

In [ ]:

# Avg queue length: 210.59
# On average, 210 customers are waiting in line at any given time

# Approximated avg wait: 68.92
# This is average time a customer spends waiting in line

# Avg wait: 210 / 3 = 70 minutes


# This is known a Litte's Law: L=λW
# L = average number in queue
# W = average wait
# λ = throughput

# It's universal — it works for:
# supermarkets
# banks
# airport security
# data centers
# call centers
# concession stands (of course)





In [145]:



import pandas as pd

results = []

# pick same arrivals from earlier
arrivals = sub["arrivals_min"].values

for servers in range(2, 11):  # 2 → 10 servers
    out = simulate_queue_discrete(
        arrivals,
        servers=servers,
        mu_per_server=1.1,
        rng=np.random.default_rng(123)
    )

    results.append({
        "servers": servers,
        "avg_queue": round(out["avg_queue"], 2),
        "avg_wait_min": round(out["avg_wait"], 2),
        "lambda_eff": round(out["lambda_eff"], 2)
    })

df_res = pd.DataFrame(results)
display(df_res)

In [ ]:
import plotly.express as px

target_wait = 5  # minutes, example SLA

fig_wait = px.line(
    df_res,
    x="servers",
    y="avg_wait_min",
    markers=True,
    title="Average Wait Time vs Number of Servers",
    labels={"servers": "Number of Servers", "avg_wait_min": "Average Wait (minutes)"}
)

# Add a horizontal line for target wait SLA
fig_wait.add_hline(
    y=target_wait,
    line_dash="dash",
    annotation_text=f"Target wait = {target_wait} min",
    annotation_position="top right"
)

fig.update_layout(height=600, width=1100)
fig_wait.show()


